In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as plt
import stock_utils as su

from signaller import Signaller
from trading_algorithm import TradingAlgorithm

from sklearn.externals import joblib

%matplotlib inline

## Choose a ticker symbol and generate a dataset with selected new features for it

In [2]:
#create a dataset for AAPL
rs = 42 # random state
tickers = ['AAPL', 'XOM', 'JNJ']
symbol = tickers[0]
window = 5
look_ahead = 1
look_behind = 3

start_date = datetime.datetime(2017, 1, 1)
end_date = datetime.datetime(2017, 10, 30)

## Load saved models

In [3]:
import os
model_filenames = []
for file in os.listdir('models'):
    if file.endswith('.pkl'):
        model_filenames.append(file)

estimators = []
for filename in model_filenames:
    print('Loading model {0}'.format(filename))
    estimators.append(joblib.load('models/{0}'.format(filename)))

Loading model ElasticNet.pkl
Loading model LinearRegression.pkl
Loading model Lasso.pkl
Loading model Ridge.pkl


## Load stock data, calculate predictions and trade signals

In [4]:
dataset = su.preprocess_data('AAPL', window, look_ahead, start_date, end_date, look_behind)
dataset.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Daily return,Rolling mean 5,Lower Bollinger band 5,Upper Bollinger band 5,Cummulative return 5,Future Price,Past Close 3
2017-01-06,115.337596,116.700551,115.031425,116.453639,0.011148,115.128215,113.581518,116.674912,0.015153,116.453639,114.715378
2017-01-07,115.337596,116.700551,115.031425,116.453639,0.000000,115.475867,113.638947,117.312787,0.015153,116.453639,114.586983
2017-01-08,115.337596,116.700551,115.031425,116.453639,0.000000,115.823519,114.049355,117.597684,0.015153,117.520300,115.169696
2017-01-09,116.493145,117.954865,116.483269,117.520300,0.009160,116.410183,114.743800,118.076566,0.025599,117.638817,116.453639
2017-01-10,117.303017,117.905483,116.838822,117.638817,0.001008,116.904007,115.667780,118.140234,0.021439,118.270913,116.453639


In [5]:
#price variation in percentage which can inform trade decisions
delta = 0.025

signaller_params = {'delta' : delta}

stock_data = []

for symbol in tickers:
    print('Processing data for {0}'.format(symbol))
    dataset = su.preprocess_data(symbol, window, look_ahead, start_date, end_date, look_behind)

    #calculate the predicted prices
    print('Calculating predictions for {0}'.format(symbol))
    pred = su.calculate_predictions_for_dataset(dataset=dataset, estimators=estimators, plot_and_print=False)
    
    #add the predicted prices to the dataset
    dataset['Predicted Price'] = pred
    
    #calculate trade signals based on different indicators and add those signals to the dataframe
    print('Calculating signals for {0}\n'.format(symbol))
    signaller = Signaller(signaller_params)

    dataset = signaller.calculate_trade_signal(dataset)
    
    stock_data.append({'name' : symbol, 'data' : dataset})

Processing data for AAPL
Calculating predictions for AAPL
Calculating signals for AAPL

Processing data for XOM
Calculating predictions for XOM
Calculating signals for XOM

Processing data for JNJ
Calculating predictions for JNJ
Calculating signals for JNJ



## Run the trading algorithm and check the results

In [6]:
trading_algo = TradingAlgorithm()
profits = trading_algo.trade(stock_data)

for p in profits:
    symbol = p['name']
    profits_df = p['profits']
    
    print('Profits for {0}'.format(symbol))
#     print(profits_df[:6])
    
    print('Normal traders mean profit: {0:.2f}%'.format(profits_df['Normal Traders'].mean()))
    print('Prediction traders mean profit: {0:.2f}%\n'.format(profits_df['Prediction Traders'].mean()))

Trading stock AAPL
Balanced Trader no predictions starts trading
Balanced Trader no predictions sells 100 stocks for 12703.26 $ on 2017-02-01 00:00:00
Balanced Trader no predictions sells 100 stocks for 12681.55 $ on 2017-02-02 00:00:00
Balanced Trader no predictions sells 100 stocks for 12735.82 $ on 2017-02-03 00:00:00
Balanced Trader no predictions buys 100 stocks for 14979.25 $ on 2017-05-17 00:00:00
Balanced Trader no predictions buys 100 stocks for 14852.63 $ on 2017-06-09 00:00:00
Balanced Trader no predictions sells 100 stocks for 15634.85 $ on 2017-08-02 00:00:00
Balanced Trader no predictions buys 100 stocks for 15354.34 $ on 2017-09-21 00:00:00
Balanced Trader no predictions sells 100 stocks for 16288.70 $ on 2017-10-27 00:00:00
After 8 transactions, Balanced Trader no predictions has 300 stocks and 103354.81$ cash spent and 70044.17$ cash made
Balanced Trader no predictions ends trading with a profit of 15.05%

Balanced Trader with predictions starts trading
Balanced Trader

In [7]:
profits[0]['profits'].head()

,Normal Traders,Prediction Traders
0,15.050528,15.050528
1,13.747478,6.447290
2,22.166360,23.889629


In [8]:
profits[1]['profits'].head()

,Normal Traders,Prediction Traders
0,0.000000,0.000000
1,-4.450448,-3.812498
2,0.292438,1.482716


In [9]:
profits[2]['profits'].head()

,Normal Traders,Prediction Traders
0,24.126019,24.126019
1,10.377686,-1.435942
2,16.757577,18.739851
